# Introduction to Reinforcement Learning

This Jupyter notebook and the others in the same folder act as supporting materials for **Chapter 22 Reinforcement Learning** of the book* Artificial Intelligence: A Modern Approach*. The notebooks make use of the implementations in `reinforcement_learning4e.py` module. We also make use of the implementation of MDPs in the `mdp4e.py` module to test our agents. It might be helpful if you have already gone through the Jupyter notebook dealing with the Markov decision process. Let us import everything from the `reinforcement_learning4e` module. It might be helpful to view the source of some of our implementations.

In [1]:
import os, sys
sys.path = [os.path.abspath("../../")] + sys.path
from reinforcement_learning4e import *

Before we start playing with the actual implementations let us review a couple of things about RL.

1. Reinforcement Learning is concerned with how software agents ought to take actions in an environment so as to maximize some notion of cumulative reward. 

2. Reinforcement learning differs from standard supervised learning in that correct input/output pairs are never presented, nor sub-optimal actions explicitly corrected. Further, there is a focus on on-line performance, which involves finding a balance between exploration (of uncharted territory) and exploitation (of current knowledge).

-- Source: [Wikipedia](https://en.wikipedia.org/wiki/Reinforcement_learning)

In summary, we have a sequence of state action transitions with rewards associated with some states. Our goal is to find the optimal policy $\pi$ which tells us what action to take in each state.

# Passive Reinforcement Learning

In passive Reinforcement Learning the agent follows a fixed policy $\pi$. Passive learning attempts to evaluate the given policy $pi$ - without any knowledge of the Reward function $R(s)$ and the Transition model $P(s'\ |\ s, a)$.

This is usually done by some method of **utility estimation**. The agent attempts to directly learn the utility of each state that would result from following the policy. Note that at each step, it has to *perceive* the reward and the state - it has no global knowledge of these. Thus, if a certain the entire set of actions offers a very low probability of attaining some state $s_+$ - the agent may never perceive the reward $R(s_+)$.

Consider a situation where an agent is given the policy to follow. Thus, at any point, it knows only its current state and current reward, and the action it must take next. This action may lead it to more than one state, with different probabilities.

For a series of actions given by $\pi$, the estimated utility $U$:
$$U^{\pi}(s) = E(\sum_{t=0}^\inf \gamma^t R^t(s'))$$
Or the expected value of summed discounted rewards until termination.

Based on this concept, we discuss three methods of estimating utility: direct utility estimation, adaptive dynamic programming, and temporal-difference learning.

### Implementation

Passive agents are implemented in `reinforcement_learning4e.py` as various `Agent-Class`es.

To demonstrate these agents, we make use of the `GridMDP` object from the `MDP` module. `sequential_decision_7x6_environment` is similar to that used for the `MDP` notebook but has discounting with $\gamma = 0.9$ and a 7x6 environment.

The `Agent-Program` can be obtained by creating an instance of the relevant `Agent-Class`. The `__call__` method allows the `Agent-Class` to be called as a function. The class needs to be instantiated with a policy ($\pi$) and an `MDP` whose utility of states will be estimated.


In [2]:
from mdp4e import sequential_decision_7x6_environment

<img src="images/grid_mdp_4x7.jpg">  

The `sequential_decision_7x6_environment` is a GridMDP object.
The rewards are **+1** and **-1** in the terminal states, and **-0.04** in the rest. Now we define actions and a policy similar to **Fig 22.1** in the book.

In [3]:
# Action Directions
to_north = (0, 1)
to_south = (0,-1)
to_west = (-1, 0)
to_east = (1, 0)

policy = {
    (0, 5): to_east,  (1, 5): to_east,  (2, 5): to_south,                 (4, 5): to_east,  (5, 5): to_east,    (6, 5): None,
    (0, 4): to_east,  (1, 4): to_east,  (2, 4): to_east, (3, 4): to_east, (4, 4): to_north,  (5, 4): to_north,  (6, 4): to_north,
    (0, 3): to_east,  (1, 3): to_east,  (2, 3): to_east, (3, 3): to_east, (4, 3): to_north,  (5, 3): to_north,  (6, 3): None,
    (0, 2): to_east,  (1, 2): to_east,  (2, 2): to_east, (3, 2): to_north, (4, 2): to_north,  (5, 2): to_north,  (6, 2): to_west,
    (0, 1): to_north,                   (2, 1): to_north,(3, 1): to_north, (4, 1): to_north,  (5, 1): to_north,  (6, 1): to_north,
    (0, 0): to_north, (1, 0): to_west,  (2, 0): to_north,(3, 0): to_north, (4, 0): to_north,  (5, 0): to_north,  (6, 0): to_north,
}
# (1, 1) and (3, 5) are blocks

This enviroment will be extensively used in the following demonstrations.

## Direct Utility Estimation (DUE)
 
 The first, most naive method of estimating utility comes from the simplest interpretation of the above definition. We construct an agent that follows the policy until it reaches the terminal state. At each step, it logs its current state, reward. Once it reaches the terminal state, it can estimate the utility for each state for *that* iteration, by simply summing the discounted rewards from that state to the terminal one.

 It can now run this 'simulation' $n$ times and calculate the average utility of each state. If a state occurs more than once in a simulation, both its utility values are counted separately.
 
 Note that this method may be prohibitively slow for very large state-spaces. Besides, **it pays no attention to the transition probability $P(s'\ |\ s, a)$.** It misses out on information that it is capable of collecting (say, by recording the number of times an action from one state led to another state). The next method addresses this issue.
 
### Examples

The `PassiveDEUAgent` class in the `reinforcement_learning4e` module implements the Agent Program described in **Fig 22.1** of the AIMA Book. `PassiveDEUAgent` sums over rewards to find the estimated utility for each state. It thus requires the running of several iterations.

In [4]:
#%psource PassiveDUEAgent

Now let's try the `PassiveDEUAgent` on the newly defined `sequential_decision_7x6_environment`:

In [5]:
DUEagent = PassiveDUEAgent(policy, sequential_decision_7x6_environment)

We can try passing information through the markove model for 200 times in order to get the converged utility value:

In [6]:
for i in range(200):
    run_single_trial(DUEagent, sequential_decision_7x6_environment)
    DUEagent.estimate_U()

Now let's print our estimated utility for each position:

In [7]:
sorted_dict = dict(sorted(DUEagent.U.items()))
print('\n'.join([str(k)+':'+str(v) for k, v in sorted_dict.items()]))

(0, 0):0.4236075354636989
(0, 1):0.5019492867686104
(0, 2):0.5486189416774652
(0, 3):0.6351350402832031
(0, 4):0.6799999999999999
(1, 0):0.25489786307017
(1, 2):0.5947449797423456
(1, 3):0.6825157148111611
(1, 4):0.6100000000000001
(1, 5):0.54
(2, 1):0.515815814336141
(2, 2):0.6350577908865893
(2, 3):0.7003470823921986
(2, 4):0.7093750000000001
(2, 5):0.7
(3, 1):0.63
(3, 2):0.6748127730419262
(3, 3):0.7192622961048641
(3, 4):0.824402309002987
(4, 2):0.6483405303955079
(4, 3):0.748238272517107
(4, 4):0.8107439163297334
(4, 5):0.8202581627145792
(5, 2):0.78
(5, 3):-0.16917236328125002
(5, 4):0.9068707057727405
(5, 5):0.928554663508546
(6, 3):-1.0
(6, 4):0.95875
(6, 5):1.0


In [8]:
print('\n'.join([str(k)+':'+str(v) for k, v in sorted(DUEagent.U.items(), key=lambda item: item[1])]))

(6, 3):-1.0
(5, 3):-0.16917236328125002
(1, 0):0.25489786307017
(0, 0):0.4236075354636989
(0, 1):0.5019492867686104
(2, 1):0.515815814336141
(1, 5):0.54
(0, 2):0.5486189416774652
(1, 2):0.5947449797423456
(1, 4):0.6100000000000001
(3, 1):0.63
(2, 2):0.6350577908865893
(0, 3):0.6351350402832031
(4, 2):0.6483405303955079
(3, 2):0.6748127730419262
(0, 4):0.6799999999999999
(1, 3):0.6825157148111611
(2, 5):0.7
(2, 3):0.7003470823921986
(2, 4):0.7093750000000001
(3, 3):0.7192622961048641
(4, 3):0.748238272517107
(5, 2):0.78
(4, 4):0.8107439163297334
(4, 5):0.8202581627145792
(3, 4):0.824402309002987
(5, 4):0.9068707057727405
(5, 5):0.928554663508546
(6, 4):0.95875
(6, 5):1.0


## Adaptive Dynamic Programming (ADP)
 
 This method makes use of knowledge of the past state $s$, the action $a$, and the new perceived state $s'$ to estimate the transition probability $P(s'\ |\ s,a)$. It does this by the simple counting of new states resulting from previous states and actions.<br> 
 The program runs through the policy a number of times, keeping track of:
    - each occurrence of state $s$ and the policy-recommended action $a$ in $N_{sa}$
    - each occurrence of $s'$ resulting from $a$ on $s$ in $N_{s'|sa}$.
     
 It can thus estimate $P(s'\ |\ s,a)$ as $N_{s'|sa}/N_{sa}$, which in the limit of infinite trials, will converge to the true value.<br>
 Using the transition probabilities thus estimated, it can apply `POLICY-EVALUATION` to estimate the utilities $U(s)$ using properties of convergence of the Bellman functions.
 
### Examples

The `PassiveADPAgent` class in the `rl` module implements the Agent Program described in **Fig 22.2** of the AIMA Book. `PassiveADPAgent` uses state transition and occurrence counts to estimate $P$, and then $U$. Go through the source below to understand the agent.

In [9]:
#%psource PassiveADPAgent

We instantiate a `PassiveADPAgent` below with the `GridMDP` shown and train it for 200 steps. The `rl` module has a simple implementation to simulate a single step of the iteration. The function is called `run_single_trial`.

In [10]:
ADPagent = PassiveADPAgent(policy, sequential_decision_7x6_environment)
for i in range(200):
    run_single_trial(ADPagent, sequential_decision_7x6_environment)

The utilities are calculated as :

In [11]:
sorted_dict = dict(sorted(ADPagent.U.items()))
print('\n'.join([str(k)+':'+str(v) for k, v in sorted_dict.items()]))

(0, 0):-0.07623967847925393
(0, 1):-0.026819464027755043
(0, 2):0.026070814424633
(0, 3):0.0898912155585346
(0, 4):0.13956118174593626
(0, 5):0.0
(1, 0):-0.11509091726010662
(1, 2):0.08520207544443542
(1, 3):0.14923809363558327
(1, 4):0.19951242416776854
(1, 5):0.1572060396703305
(2, 0):0.0
(2, 1):0.08676528114433374
(2, 2):0.1453176451935049
(2, 3):0.21493318655409463
(2, 4):0.28790869095629196
(2, 5):0.2191178218559228
(3, 0):0.0
(3, 1):0.15696301474218163
(3, 2):0.21884779416286657
(3, 3):0.3007460686047139
(3, 4):0.37928397029167676
(4, 0):0.0
(4, 1):0.0
(4, 2):0.3170521270289436
(4, 3):0.40798932594936077
(4, 4):0.5114264677843369
(4, 5):0.6463636112746671
(5, 0):0.0
(5, 1):0.0
(5, 2):0.38091858032242826
(5, 3):0.5226408411964897
(5, 4):0.6387294379851319
(5, 5):0.8108590985010455
(6, 0):0.0
(6, 1):0.0
(6, 2):0.3028267222879898
(6, 3):0.0
(6, 4):0.7064549063271298
(6, 5):1.0


In [12]:
print('\n'.join([str(k)+':'+str(v) for k, v in sorted(ADPagent.U.items(), key=lambda item: item[1])]))

(1, 0):-0.11509091726010662
(0, 0):-0.07623967847925393
(0, 1):-0.026819464027755043
(4, 0):0.0
(5, 1):0.0
(0, 5):0.0
(3, 0):0.0
(5, 0):0.0
(6, 1):0.0
(4, 1):0.0
(2, 0):0.0
(6, 0):0.0
(6, 3):0.0
(0, 2):0.026070814424633
(1, 2):0.08520207544443542
(2, 1):0.08676528114433374
(0, 3):0.0898912155585346
(0, 4):0.13956118174593626
(2, 2):0.1453176451935049
(1, 3):0.14923809363558327
(3, 1):0.15696301474218163
(1, 5):0.1572060396703305
(1, 4):0.19951242416776854
(2, 3):0.21493318655409463
(3, 2):0.21884779416286657
(2, 5):0.2191178218559228
(2, 4):0.28790869095629196
(3, 3):0.3007460686047139
(6, 2):0.3028267222879898
(4, 2):0.3170521270289436
(3, 4):0.37928397029167676
(5, 2):0.38091858032242826
(4, 3):0.40798932594936077
(4, 4):0.5114264677843369
(5, 3):0.5226408411964897
(5, 4):0.6387294379851319
(4, 5):0.6463636112746671
(6, 4):0.7064549063271298
(5, 5):0.8108590985010455
(6, 5):1.0


When comparing to the result of `PassiveDUEAgent`, they both have -1.0 for utility at (3,1) and 1.0 at (3,2). Another point to notice is that the spot with the highest utility for both agents is (2,2) beside the terminal states, which is easy to understand when referring to the map.

## Temporal-difference learning (TD)
 
 Instead of explicitly building the transition model $P$, the temporal-difference model makes use of the expected closeness between the utilities of two consecutive states $s$ and $s'$.
 For the transition $s$ to $s'$, the update is written as:
$$U^{\pi}(s) \leftarrow U^{\pi}(s) + \alpha \left( R(s) + \gamma U^{\pi}(s') - U^{\pi}(s) \right)$$
 This model implicitly incorporates the transition probabilities by being weighed for each state by the number of times it is achieved from the current state. Thus, over a number of iterations, it converges similarly to the Bellman equations.
 The advantage of the TD learning model is its relatively simple computation at each step, rather than having to keep track of various counts.
 For $n_s$ states and $n_a$ actions the ADP model would have $n_s \times n_a$ numbers $N_{sa}$ and $n_s^2 \times n_a$ numbers $N_{s'|sa}$ to keep track of. The TD model must only keep track of a utility $U(s)$ for each state.
 
### Examples

`PassiveTDAgent` uses temporal differences to learn utility estimates. We learn the difference between the states and back up the values to previous states.  Let us look into the source before we see some usage examples.

In [13]:
#%psource PassiveTDAgent

In creating the `TDAgent`, we use the **same learning rate** $\alpha$ as given in the footnote of the book: $\alpha(n)=60/(59+n)$

In [14]:
TDagent = PassiveTDAgent(policy, sequential_decision_7x6_environment, alpha = lambda n: 60./(59+n))

Now we run **200 trials** for the agent to estimate Utilities.

In [15]:
for i in range(200):
    run_single_trial(TDagent,sequential_decision_7x6_environment)

The calculated utilities are:

In [16]:
sorted_dict = dict(sorted(TDagent.U.items()))
print('\n'.join([str(k)+':'+str(v) for k, v in sorted_dict.items()]))

(0, 0):-0.06816744339324231
(0, 1):-0.040562528263606706
(0, 2):-0.013493772802435557
(0, 3):-0.05069748077431516
(0, 4):0.17640258872909165
(0, 5):0.0
(1, 0):-0.12274055298989325
(1, 2):0.08441324404957194
(1, 3):0.08485166602673662
(1, 4):0.10832713351599019
(1, 5):0.1846696109053968
(2, 0):0.0
(2, 1):0.0669990568349042
(2, 2):0.1374297077160365
(2, 3):0.2456154066636996
(2, 4):0.31538811836991154
(2, 5):0.20878101109686703
(3, 0):0.0
(3, 1):0.14457846345708605
(3, 2):0.23920553069402883
(3, 3):0.352416704597033
(3, 4):0.42801488656630904
(4, 0):0.0
(4, 1):0.0
(4, 2):0.34481095007523177
(4, 3):0.20316139389877338
(4, 4):0.5327401364248504
(4, 5):0.6519972375057037
(5, 0):0.0
(5, 1):0.0
(5, 2):0.25051847776494396
(5, 3):0.2927954280996534
(5, 4):0.5982770886599733
(5, 5):0.8508474454186945
(6, 0):0.0
(6, 1):0.0
(6, 2):-0.10294029383053863
(6, 3):-1
(6, 4):0.844325539872483
(6, 5):1


In [17]:
print('\n'.join([str(k)+':'+str(v) for k, v in sorted(TDagent.U.items(), key=lambda item: item[1])]))

(6, 3):-1
(1, 0):-0.12274055298989325
(6, 2):-0.10294029383053863
(0, 0):-0.06816744339324231
(0, 3):-0.05069748077431516
(0, 1):-0.040562528263606706
(0, 2):-0.013493772802435557
(4, 0):0.0
(5, 1):0.0
(0, 5):0.0
(3, 0):0.0
(5, 0):0.0
(6, 1):0.0
(4, 1):0.0
(2, 0):0.0
(6, 0):0.0
(2, 1):0.0669990568349042
(1, 2):0.08441324404957194
(1, 3):0.08485166602673662
(1, 4):0.10832713351599019
(2, 2):0.1374297077160365
(3, 1):0.14457846345708605
(0, 4):0.17640258872909165
(1, 5):0.1846696109053968
(4, 3):0.20316139389877338
(2, 5):0.20878101109686703
(3, 2):0.23920553069402883
(2, 3):0.2456154066636996
(5, 2):0.25051847776494396
(5, 3):0.2927954280996534
(2, 4):0.31538811836991154
(4, 2):0.34481095007523177
(3, 3):0.352416704597033
(3, 4):0.42801488656630904
(4, 4):0.5327401364248504
(5, 4):0.5982770886599733
(4, 5):0.6519972375057037
(6, 4):0.844325539872483
(5, 5):0.8508474454186945
(6, 5):1


When comparing to previous agents, the result of `PassiveTDAgent` is closer to `PassiveADPAgent`.